In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "/data/malu/Qwen2.5-0.5B-Instruct"  # 可替换为其他自回归模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

/data/malu/miniconda3/envs/dynamic_rl_sampler/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Token: ĠParis   | Probability: 0.3022


In [ ]:
def batch_calculate_probs(question, answer):
    # 编码时不添加特殊token
    question_ids = tokenizer.encode(question, add_special_tokens=False, return_tensors="pt")
    answer_ids = tokenizer.encode(answer, add_special_tokens=False, return_tensors="pt")
    
    # 拼接完整序列
    full_ids = torch.cat([question_ids, answer_ids], dim=1)
    
    # 前向传播获取所有logits
    with torch.no_grad():
        outputs = model(full_ids)
    
    # 提取概率
    logits = outputs.logits[0]  # 取第一个样本
    probs = torch.softmax(logits, dim=-1)
    
    # 计算答案部分概率
    start_pos = question_ids.shape[1] - 1  # 起始位置
    answer_probs = []
    
    for i in range(answer_ids.shape[1]):
        pos = start_pos + i
        token_id = answer_ids[0, i].item()
        answer_probs.append(probs[pos, token_id].item())
    
    # 转换token
    tokens = tokenizer.convert_ids_to_tokens(answer_ids[0])
    return list(zip(tokens, answer_probs))

# 使用示例
question = "The capital of France is"
answer = " Paris"
result = batch_calculate_probs(question, answer)

for token, prob in result:
    print(f"Token: {token:<8} | Probability: {prob:.4f}")

In [2]:
import pandas as pd

input_file = "../dataset/sub_openr1.parquet"
df = pd.read_parquet(input_file)
df.head()

,data_source,prompt,target,ability,reward_model,extra_info
0,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, so I need to find ...",,"{'ground_truth': 'v_{R}=4\mathrm{~}/\mathrm{},...","{'index': -1, 'split': 'default'}"
1,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, let me try to figu...",,"{'ground_truth': '180', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
2,cn_contest,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, so I have this pro...",,"{'ground_truth': 'D', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
3,olympiads,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, so I need to solve...",,"{'ground_truth': '\frac{1}{4}', 'style': 'rule'}","{'index': -1, 'split': 'default'}"
4,aops_forum,[{'content': 'Your task is to follow a systema...,"[{'content': '<think> Okay, so I need to find ...",,"{'ground_truth': '-\ln 2', 'style': 'rule'}","{'index': -1, 'split': 'default'}"


In [3]:
for i in range(len(df)):
    sample = df.iloc[i]

    message = sample['prompt']
    target = sample['target'][0]["content"]

    prompt_with_chat_template = tokenizer.apply_chat_template(
                    message,
                    tokenize=False,
                    add_generation_prompt=True
                )

    if prompt_with_chat_template.endswith('<think>\n') and target.startswith('<think>\n'):
        target = target[len('<think>\n'):]

    result = batch_calculate_probs(prompt_with_chat_template, target)
    for token, prob in result:
        print(f"Token: {token:<8} | Probability: {prob:.4f}")

<|im_start|>system
Your task is to follow a systematic, thorough reasoning process before providing the final solution. This involves analyzing, summarizing, exploring, reassessing, and refining your thought process through multiple iterations. Structure your response into two sections: Thought and Solution. In the Thought section, present your reasoning using the format: "<think>
 {thoughts} </think>
". Each thought should include detailed analysis, brainstorming, verification, and refinement of ideas. After "</think>
," in the Solution section, provide the final, logical, and accurate answer, clearly derived from the exploration in the Thought section. If applicable, include the answer in \boxed{} for closed-form results like multiple choices or mathematical solutions.<|im_end|>
<|im_start|>user
## Task B-1.3.

A ship traveling along a river has covered $24 \mathrm{~km}$ upstream and $28 \mathrm{~km}$ downstream. For this journey, it took half an hour less than for traveling $30 \mat